In [2]:
%pip install tensorboardX --user
%pip install bs4 --user
%pip install plotly --user
%pip install -U auquan_toolbox --user


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install auquan-toolbox --user
%pip show auquan-toolbox

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: six>=1.5 in d:\programming\anaconda\lib\site-packages (from python-dateutil>=2.8.1->pandas->auquan-toolbox) (1.16.0)



Name: auquan-toolbox
Version: 2.1.92
Summary: Auquan Toolbox for developing strategies and backtesting
Home-page: https://auquan.com
Author: Auquan Team
Author-email: info@auquan.com
Note: you may need to restart the kernel to use updated packages.


License: MIT
Location: c:\users\hp\appdata\roaming\python\python38\site-packages
Requires: numpy, lxml, tensorboardX, pandas, sklearn, requests, tensorboard, bs4, scipy, plotly
Required-by: 


In [4]:
import sys
sys.path.append(r'c:\users\hp\appdata\roaming\python\python38\site-packages')
print(sys.path)

['c:\\Users\\hp\\Documents\\GitHub\\trading x ml', 'd:\\PROGRAMMING\\anaconda\\python38.zip', 'd:\\PROGRAMMING\\anaconda\\DLLs', 'd:\\PROGRAMMING\\anaconda\\lib', 'd:\\PROGRAMMING\\anaconda', '', 'C:\\Users\\hp\\AppData\\Roaming\\Python\\Python38\\site-packages', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages\\locket-0.2.1-py3.8.egg', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages\\win32', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages\\win32\\lib', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages\\Pythonwin', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hp\\.ipython', 'c:\\users\\hp\\appdata\\roaming\\python\\python38\\site-packages']


In [3]:
from backtester.trading_system_parameters import TradingSystemParameters
from backtester.features.feature import Feature
from backtester.timeRule.custom_time_rule import CustomTimeRule
from backtester.executionSystem.simple_execution_system import SimpleExecutionSystem
from backtester.orderPlacer.backtesting_order_placer import  BacktestingOrderPlacer
from backtester.trading_system import TradingSystem
from backtester.dataSource.nse_data_source import NSEStockDataSource
from backtester.constants import *


In [4]:
class TradeParameters(TradingSystemParameters):

    def __init__(self,tradefns):
        self.__tradefns=tradefns
        super(TradeParameters,self).__init__()
        self.__dataSetID = 'equity_data'
        self.__instrumentIds = self.__tradefns.getSymbolsToTrade()
        self.__startDate = '2015/01/02'
        self.__endDate = '2017/08/31'

    def getDataParser(self):
        instrumentIds = self.__tradefns.getSymbolsToTrade()
        return NSEStockDataSource( cachedFolderName = 'historicalData/',
            dataSetId = self.__dataSetId,
            instrumentIds = instrumentIds,
            startDateStr = self.__startDate,
            endDateStr = self.__endDate,)
    
    def getTimeRuleForUpdates(self):
        return CustomTimeRule(
            startDate =self.__startDate,
            endDate = self.__endDate,
            frequency = 'D',
            sample= '30'
        )
    
    def getFrequencyOffeatureUpdates():
        return timedelta( days =1)
    
    def getStartingCapital(self):
        if len(self.__tradefns.getSymbolsToTrade()) > 0:
            return 1000*len(self.__tradefns.getSymbolsToTrade())
        else:
            return 30000
        
    def getCustomFeatures(self):
        return  {
            'my_custom_feature': MyCustomFeature,
            'prediction': TrainingPredictionFeature,
            'zero_fees': FeesCalculator,
            'benchmark_PnL': BuyHoldPnL,
            'score': ScoreFeature
        }
    def getInstrumentFeatureConfigDicts(self):
        
        predictionDict = {
            'featureKey': 'prediction',
            'featureId': 'prediction',
            'params': {}
        }
        feesConfigDict = {
            'featureKey': 'fees',
            'featureId': 'zero_fees',
            'params': {}
        }
        profitlossConfigDict = {
            'featureKey': 'pnl',
            'featureId': 'pnl',
            'params': {
                'price': self.getPriceFeatureKey(),
                'fees': 'fees'
            }
        }
        capitalConfigDict = {
            'featureKey': 'capital',
            'featureId': 'capital',
            'params': {
                'price': 'adjClose',
                'fees': 'fees',
                'capitalReqPercent': 0.95
            }
        }
        benchmarkDict = {
            'featureKey': 'benchmark',
            'featureId': 'benchmark_PnL',
            'params': {'pnlKey': 'pnl'}
        }
        scoreDict = {
            'featureKey': 'score',
            'featureId': 'score',
            'params': {
                'featureName1': 'pnl',
                'featureName2':'benchmark'
            }
        }

        stockFeatureConfigs = self.__tradingFunctions.getInstrumentFeatureConfigDicts()

        return {
            INSTRUMENT_TYPE_STOCK: stockFeatureConfigs + [
                predictionDict,
                feesConfigDict,
                profitlossConfigDict,
                capitalConfigDict,
                benchmarkDict, 
                scoreDict
            ]
        }
    
    def getMarketFeatureConfigDicts(self):
          scoreDict = {
            'featureKey': 'score',
            'featureId': 'score_ll',
            'params': {
                'featureName': self.getPriceFeatureKey(),
                'instrument_score_feature': 'score'
            }
        }
          return [scoreDict]

    def getPrediction(self, time, updateNum, instrumentManager):
        predictions = pd.Series(index = self.__instrumentIds)
        predictions = self.__tradefns.getPrediction(time, updateNum, instrumentManager, predictions)
        return predictions
    
    def getExecutionSystem(self):
            return SimpleExecutionSystem(
            enter_threshold = 0.7,
            exit_threshold = 0.55,
            longLimit = 1,
            shortLimit = 1,
            capitalUsageLimit = 0.10*self.getStartingCapital(),
            enterlotSize = 1, 
            exitlotSize = 1,
            limitType = 'L', 
            price = 'adjClose'
        )
    def getOrderPlacer(self):
        return BacktestingOrderPlacer
    
    def getLookbackSize(self):
        return 120
    
    def getPriceFeatureKey(self):
        return 'adjClose'
    
    def getInstrumentsIds(self):
        
        return self.__instrumentIds



In [8]:
class TrainingPredictionFeature(Feature):
    @classmethod
    def computeInstruments(cls,updateNum,time,featureParams,featureKey,instrumentManager):
        tf= MyTradingFunctions()
        tp=MyTradingParams(tf)

class FeeCalc(Feature):
    @classmethod
    def computeforinstruments(cls,updateNum,time,featureParams,featureKey,instrumentManager):
        return pd.Series(0,index=instrumentManager.getAllInstrumentId())
    
class BuyHoldPnL(Feature):
    
    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        instrumentLookbackData = instrumentManager.getLookbackInstrumentFeatures()

        priceData = instrumentLookbackData.getFeatureDf('adjClose')
         
        if len(priceData) < 2:
            return pd.Series(0, index = instrumentManager.getAllInstrumentsByInstrumentId())
        else:
            bhpnl = instrumentLookbackData.getFeatureDf(featureKey).iloc[-1]
            bhpnl += priceData.iloc[-1] - priceData.iloc[-2]

        return bhpnl
    
    
class ScoreFeature(Feature):
    
    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        instrumentLookbackData = instrumentManager.getLookbackInstrumentFeatures()
        if len(instrumentLookbackData.getFeatureDf(featureParams['featureName1'])) > 0:
            feature1 = instrumentLookbackData.getFeatureDf(featureParams['featureName1']).iloc[-1]
            feature2 = instrumentLookbackData.getFeatureDf(featureParams['featureName2']).iloc[-1]
            
            for instrumentId in feature1.index:
                pnls = instrumentLookbackData.getFeatureDf('pnl')[instrumentId]
                positions = instrumentLookbackData.getFeatureDf('position')[instrumentId]
                
                print(instrumentId)
                print('pnl: %.2f'%pnls[-1])
                if len(positions) > 2 and np.abs(positions[-1] - positions[-2]) > 0:
                    print('Position changed to: %.2f'%positions[-1])
            
            toRtn = (feature1 - feature2) / feature2.abs()
            toRtn[toRtn.isnull()] = 0
            toRtn[toRtn == np.Inf] = 0
        else:
            toRtn=0

        return toRtn


: 